In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
X,y = mnist["data"], mnist["target"]
X.shape

(70000, 784)

In [3]:
y.shape

(70000,)

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt

some_digit = X[0]
some_digit_image = some_digit.reshape(28,28)

plt.imshow(some_digit_image, cmap = mpl.cm.binary, interpolation = "nearest")
plt.axis("off")
plt.show()

In [5]:
y[0]

'5'

In [6]:
import numpy as np

y = y.astype(np.uint8)

In [7]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [8]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

In [9]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state= 42)
sgd_clf.fit(X_train, y_train_5)
sgd_clf.predict([some_digit])

array([ True])

In [11]:
from sklearn.metrics import accuracy_score

y_pred = sgd_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.09241666666666666

In [15]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

svm_clf = SVC(decision_function_shape="ovr", gamma="auto")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])


y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9449166666666666

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=2.9592983686234353, gamma=0.012606827538172187 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . C=2.9592983686234353, gamma=0.012606827538172187, total=   1.5s
[CV] C=2.9592983686234353, gamma=0.012606827538172187 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV] . C=2.9592983686234353, gamma=0.012606827538172187, total=   1.5s
[CV] C=2.9592983686234353, gamma=0.012606827538172187 ................
[CV] . C=2.9592983686234353, gamma=0.012606827538172187, total=   1.5s
[CV] C=3.157234736321602, gamma=0.021908383907339252 .................
[CV] .. C=3.157234736321602, gamma=0.021908383907339252, total=   1.5s
[CV] C=3.157234736321602, gamma=0.021908383907339252 .................
[CV] .. C=3.157234736321602, gamma=0.021908383907339252, total=   1.5s
[CV] C=3.157234736321602, gamma=0.021908383907339252 .................
[CV] .. C=3.157234736321602, gamma=0.021908383907339252, total=   1.7s
[CV] C=3.1119327718994363, gamma=0.014871184754431391 ................
[CV] . C=3.1119327718994363, gamma=0.014871184754431391, total=   2.0s
[CV] C=3.1119327718994363, gamma=0.014871184754431391 ................
[CV] . C=3.1119327718994363, gamma=0.014871184754431391, total=   1.8s
[CV] C=3.1119327718994363, gamma=0.014871184754431391 ................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   49.2s finished


RandomizedSearchCV(cv=3, estimator=SVC(gamma='auto'),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F65EBDC400>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F65EBD8F60>},
                   verbose=2)

In [23]:
rnd_search_cv.best_estimator_


SVC(C=2.904950117504194, gamma=0.002555579589372341)

In [22]:
rnd_search_cv.best_score_


0.8399836962711213

In [19]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)


SVC(C=2.904950117504194, gamma=0.002555579589372341)

In [20]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9988833333333333

In [21]:

y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9684

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv = 3, scoring="accuracy")

array([0.95035, 0.96035, 0.9604 ])

In [15]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self,X):
        return np.zeros((len(X),1),dtype=bool)

In [17]:
never5clf= Never5Classifier()
cross_val_score(never5clf, X_train, y_train_5, cv=3, scoring="accuracy")

array([0.91125, 0.90855, 0.90915])

In [18]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

array([[53892,   687],
       [ 1891,  3530]], dtype=int64)

In [20]:
y_train_perfect_pred = y_train_5
confusion_matrix(y_train_5, y_train_perfect_pred)

array([[54579,     0],
       [    0,  5421]], dtype=int64)

In [22]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

0.8370879772350012

In [23]:
recall_score(y_train_5, y_train_pred)

0.6511713705958311

In [24]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)

0.7325171197343846